## Get info of each tag
Here are some examples of responded API:
- https://api.rawg.io/api/games/grand-theft-auto-v
- https://rawg.io/games/grand-theft-auto-v

In [2]:
import csv
import requests
import json
from pprint import pprint
from time import time
import concurrent.futures
import functools
import os

### Load CSV file which has tag's id and its name

In [3]:
csv_data = []
with open("data/tag_id.csv", "r") as f:
    csv_data = list(csv.reader(f))
for i, val in enumerate(csv_data):
    print(val)
    if i==10: break

['31', 'singleplayer']
['40847', 'steam-achievements']
['7', 'multiplayer']
['7808', 'steam-trading-cards']
['40849', 'steam-cloud']
['13', 'atmospheric']
['40836', 'full-controller-support']
['42', 'great-soundtrack']
['24', 'rpg']
['18', 'co-op']
['170', 'competitive']


## Multithreading

### This function is responsible for requesting each tag and save as a JSON file in `/data/tag_info/`

In [4]:
def threading(start_index, tags_per_worker, urls, downloaded_files, headers):
    for url in urls[start_index : start_index + tags_per_worker]:
        if url.rsplit("/")[-1] in downloaded_files: continue
        try:
            # Request API
            json_data = json.loads(requests.get(url, headers=headers).text)

            # Only include wanted keys
            D = {k:v for k,v in json_data.items() if k in include}
            
            # Save as JSON file
            name = D["id"]
            with open(f"data/tag_info/{name}.json","w", encoding="utf-8") as f:
                json.dump(D, f)
        except:
            print(f"Error with {url}")

#### Create folder if not existed

In [5]:
import os
if not os.path.exists('data/tag_info/'):
    os.makedirs('data/tag_info/')

#### Threading Preparation

In [6]:
headers = { 'User-Agent': 'App Name: Education purpose',}
include = {
    "esrb_rating",        
           "id",
           "slug",
           "name",
           "games_count",
           "description",
           "language"
          }

# Set up number of workers
max_workers = 100
start_tag_index = 0 # min: 0
end_tag_index = len(csv_data) # max: 345707
number_of_tags = end_tag_index - start_tag_index
tags_per_worker = int(number_of_tags/max_workers) 
start_index = range(start_tag_index, end_tag_index, tags_per_worker)

# Make urls
base_url = "https://api.rawg.io/api/tags/"
key = ['?key=13961a85724f47818f318bebae6b82e1' for i in range(len(csv_data)) ]
urls = [base_url + csv_data[i][0] +key[i] for i in range(len(csv_data))]

In [7]:
urls[:10]

['https://api.rawg.io/api/tags/31?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/40847?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/7?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/7808?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/40849?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/13?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/40836?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/42?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/24?key=13961a85724f47818f318bebae6b82e1',
 'https://api.rawg.io/api/tags/18?key=13961a85724f47818f318bebae6b82e1']

In [8]:
number_of_tags

3250

In [9]:
# Skipped downloaded files
downloaded_files = {file.split(".",1)[0] for file in os.listdir("data/tag_info/")}

# Time
t0 = time()
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    temp = functools.partial(threading,
                             tags_per_worker=tags_per_worker,
                             urls=urls,
                             downloaded_files=downloaded_files,
                             headers=headers,
                            )
    executor.map(temp, start_index)
    
# Time diff
print(f"Time taken: {time()-t0}")

Error with https://api.rawg.io/api/tags/944?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/tags/2267?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/tags/2889?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/tags/5357?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/tags/2443?key=13961a85724f47818f318bebae6b82e1
Time taken: 41.16357088088989
